## Définir la stratégie de collecte de données
En recensant les API disponibles, et réaliser la collecte des données répondant à des critères définis via une API (interface de programmation) en prenant en compte les normes RGPD, afin de les exploiter pour l’élaboration d’un modèle.

### CE1: Stratégie de Collecte de Données
Vous avez défini sa stratégie de collecte de données et recensé les API disponibles pour le besoin de votre projet.

### CE2: Requête de l'API
Vous avez écrit et testé une requête pour obtenir les données via l’API.

### CE3: Sélection des Champs Nécessaires
Vous avez récupéré les seuls champs nécessaires. Dans le cadre de ce projet, il s’agit par exemple :
- Des champs qui ont servi de filtre dans la requête SQL.
- Des champs utiles au traitement de proposition de mots clés : `Title`, `Body`, `Tags`.

### CE4: Application des Filtres
Vous avez appliqué au moins un filtre sur un des champs nécessaires pour ne collecter que les données ayant les valeurs correspondantes sur ce ou ces champs.

### CE5: Stockage des Données
Vous avez stocké les données collectées via l’API dans un fichier utilisable (ex. : fichier CSV ou pickle).

### CE6: Respect des Normes RGPD
Vous avez veillé au respect des normes RGPD dans toutes les phases de la collecte et du stockage des données. En particulier, il présente les 5 grands principes du RGPD et sa mise en oeuvre sur le projet :
- Il ne gère que les données nécessaires pour la finalité du projet.
- Il ne gère aucune information permettant d’identifier les personnes qui sont les auteurs des questions.


In [4]:
import os
from stackapi import StackAPI
import pandas as pd
from datetime import datetime

ROOT_PATH = os.getcwd()
DATA_SAVE_FILE = os.path.join(ROOT_PATH, "data/StackOverFlow.csv.gz")

# Initialize key
key = "ub*oRqta6kWgck7l2tG5ng(("

# Initialize StackAPI
SITE = StackAPI('stackoverflow', key = key)

# Define Query Parameters
tag = None
min_score = 50
from_date = datetime.strptime('2010-01-01', '%Y-%m-%d').timestamp()
to_date = datetime.strptime('2023-01-31', '%Y-%m-%d').timestamp()

# Set the max pagesize
max_pagesize = 100

# Initialize list to store question data
questions_data = []

# Fetch Questions with pagination up to page 25
is_fetching = True
page = 1
max_page = 25  # Set the maximum page to 25

while is_fetching:
    questions = SITE.fetch('questions', 
                           tagged=tag, 
                           min=min_score, 
                           fromdate=int(from_date), 
                           todate=int(to_date), 
                           sort='votes', 
                           order='desc', 
                           pagesize=max_pagesize, 
                           page=page,)
    for question in questions['items']:
        date = datetime.fromtimestamp(question['creation_date'])
        title = question['title']
        tags = ', '.join(question['tags'])
        score = question['score']
        questions_data.append({'Date': date, 'Title': title, 'Tags': tags, 'Score': score})
    
    page += 1
    is_fetching = 'has_more' in questions and questions['has_more']


# Create DataFrame from list
df = pd.DataFrame(questions_data, columns=['Date', 'Title', 'Tags', 'Score'])

df.to_csv(DATA_SAVE_FILE, compression="gzip", index=False)

# Display DataFrame
df



,Date,Title,Tags,Score
0,2012-06-27 15:51:36,Why is processing a sorted array faster than p...,"java, c++, performance, cpu-architecture, bran...",27139
1,2010-01-05 02:12:15,How do I delete a Git branch locally and remot...,"git, version-control, git-branch, git-push, gi...",20351
2,2011-04-24 00:17:18,How can I remove a specific item from an array...,"javascript, arrays",11702
3,2011-07-06 05:20:36,How can I rename a local Git branch?,"git, version-control, git-branch",11429
4,2011-11-29 23:54:22,Why does HTML think “chucknorris” is a color?,"html, browser, background-color",8721
...,...,...,...,...
553434,2022-05-21 03:28:00,cannot import name &#39;pad_sequences&#39; fro...,"python, keras, python-import",50
553435,2022-07-06 10:16:39,Unhandled Exception: PlatformException(channel...,"flutter, dart",50
553436,2022-08-10 09:49:30,CreationExtras must have a value by `SAVED_STA...,"android, kotlin, android-jetpack-compose, andr...",50
553437,2022-09-02 14:39:13,mongo: command not found on mongodb 6.0 docker...,"mongodb, docker",50
